# 测试模型估计的准确性

在贝叶斯模型设计程序中, 我们编写了多个测试模型.

1. 经典 Tim Behrens 模型, 仅在 y 改为多维度的 bernulli 分布
2. 将 y 的分布改为 mulitnormal 分布, 随后将 r 的分布改为狄利克雷分布
3. 将 y 按照不同的信息维度分为多个 bernulli 分布, 同时对应的 r 有多个 beta 分布.

模型1的可靠性很低, 同时今天 (2020-07-05) 已经测试估计了模型3的结果

In [11]:
library(tidyverse)
library(here)

sub_data <- read.csv(here("data", "sub01_Yangmiao_s.csv"))
estimated_data_model3 <- read.csv(here("data", "output", "multi_dim_bayesian_learner", "model3.csv"))
estimated_data_model1 <- read.csv(here("data", "output", "multi_dim_bayesian_learner", "model1.csv"))

In [5]:
head(estimated_data_model3)
head(sub_data)

,X,k_list,v_list,r1_list,r2_list,k_cap,v_cap
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,-1.2255343,0.7230516,0.8217114,0.15118530,2.721202,3.0878398
2,1,-0.9161216,0.4184744,0.9227206,0.05688571,9.044193,2.2073483
3,2,-0.6407631,-0.1191336,0.9442959,0.54538256,4.331058,1.5274851
4,3,-0.7902760,-0.6214576,0.9542452,0.67905850,5.270791,0.7957149
5,4,-0.5940357,-1.1110465,0.9618944,0.78128095,2.989975,0.4820522
6,5,-0.8094041,-1.9001080,0.9694178,0.83584244,2.912134,0.3016363


,Subject,color,location,Response,contigency,RT,Type,condition,Time,index,cue,reward
,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>
1,sub01_Yangmiao,green,left,2,inc,554.8,hit,s,4292,481,0,1
2,sub01_Yangmiao,green,left,2,inc,381.7,hit,s,47621,482,0,1
3,sub01_Yangmiao,red,left,1,con,530.7,hit,s,98449,483,0,1
4,sub01_Yangmiao,red,left,1,con,423.7,hit,s,149278,484,0,1
5,sub01_Yangmiao,red,left,1,con,348.7,hit,s,200106,485,0,1
6,sub01_Yangmiao,red,left,1,con,311.7,hit,s,243435,486,0,1


在估计的数据中, r1 / y1 对应的是空间位置 (location), r2 / y2 对应的是动作. 对应关系 {'left':1, 'right':0}

因此我们可以将估计得到的结果整理如下表:

In [14]:
p_left_loc <- estimated_data_model3$r1_list
p_right_loc <- 1 - estimated_data_model3$r1_list
p_left_hand <- estimated_data_model3$r2_list
p_right_hand <- 1 - estimated_data_model3$r2_list

probability_table <- data.frame(p_left_loc, p_right_loc, p_left_hand, p_right_hand)

head(probability_table)

,p_left_loc,p_right_loc,p_left_hand,p_right_hand
,<dbl>,<dbl>,<dbl>,<dbl>
1,0.8217114,0.17828865,0.15118530,0.8488147
2,0.9227206,0.07727939,0.05688571,0.9431143
3,0.9442959,0.05570415,0.54538256,0.4546174
4,0.9542452,0.04575485,0.67905850,0.3209415
5,0.9618944,0.03810560,0.78128095,0.2187190
6,0.9694178,0.03058216,0.83584244,0.1641576


如此我们首先可以计算 con/inc 的概率. 

$$ P(con) = P(loc_{left}) * P(hand_{left}) + P(loc_{right}) * P(hand_{right}) $$

$$ P(inc) = P(loc_{left}) * P(hand_{right}) + P(loc_{right}) * P(hand_{left}) $$

In [9]:
p_con <- p_left_loc * p_left_hand + p_right_loc * p_right_hand
p_inc <- p_left_loc * p_right_hand + p_right_loc * p_left_hand

probability_table <- cbind(probability_table, p_con, p_inc)

同时也可以计算不同的刺激反应联结的概率:

In [17]:
p_left_loc_left_hand <- p_left_loc * p_left_hand
p_left_loc_right_hand <- p_left_loc * p_right_hand
p_right_loc_left_hand <- p_right_loc * p_left_hand
p_right_loc_right_hand <- p_right_loc * p_right_hand

probability_table <- cbind(probability_table, p_left_loc_left_hand, p_left_loc_right_hand, 
                                              p_right_loc_left_hand, p_right_loc_right_hand)

In [18]:
head(probability_table)

,p_left_loc,p_right_loc,p_left_hand,p_right_hand,p_left_loc_left_hand,p_left_loc_right_hand,p_right_loc_left_hand,p_right_loc_right_hand
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.8217114,0.17828865,0.15118530,0.8488147,0.12423068,0.6974807,0.026954623,0.151334025
2,0.9227206,0.07727939,0.05688571,0.9431143,0.05248962,0.8702310,0.004396093,0.072883301
3,0.9442959,0.05570415,0.54538256,0.4546174,0.51500249,0.4292934,0.030380070,0.025324076
4,0.9542452,0.04575485,0.67905850,0.3209415,0.64798828,0.3062569,0.031070218,0.014684629
5,0.9618944,0.03810560,0.78128095,0.2187190,0.75150977,0.2103846,0.029771182,0.008334421
6,0.9694178,0.03058216,0.83584244,0.1641576,0.81028057,0.1591373,0.025561871,0.005020293


假设被试的行为反应完全按照概率来执行, 此时可以做出被试对应的行为反应结果:

In [54]:
make_choice <- function(options_prob_table){
    choice_results <- data.frame(nrow = nrow(options_prob_table))
    options_item <- colnames(options_prob_table)
    for(i in 1:nrow(options_prob_table)){
        options_vector <- options_prob_table[i,]
        choice_results[i,] <- options_item[which(options_vector == max(options_vector))]
    }
    
    return(choice_results)
}

In [55]:
make_choice(probability_table[, 5:8]) %>% 
    tidyr::separate(col = nrow, into = as.character(1:5), sep = "_") %>% 
    select(`4`) %>% 
    transmute(Response = case_when(`4` == "right" ~ 2, `4` == "left" ~ 1)) -> choices

In [60]:
cor.test(choices$Response, sub_data$Response)


	Pearson's product-moment correlation

data:  choices$Response and sub_data$Response
t = 16.402, df = 478, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.5395846 0.6544542
sample estimates:
      cor 
0.6001042 
